In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

# ============================
# 1. CSV の読み込み
# ============================
# 日本語データの多くは Shift-JIS / CP932 なのでまず cp932 で読む
df = pd.read_csv("work_dataset/games.csv", encoding="cp932")

# ============================
# 2. 勝敗判定用の列を作る
# ============================
df["winner"] = None
df["loser"] = None

# 引き分け判定
df["is_draw"] = df["home_team_score"] == df["away_team_score"]

# ホーム勝利
home_win = df["home_team_score"] > df["away_team_score"]
df.loc[home_win, "winner"] = df.loc[home_win, "home_team_id"]
df.loc[home_win, "loser"]  = df.loc[home_win, "away_team_id"]

# アウェイ勝利
away_win = df["home_team_score"] < df["away_team_score"]
df.loc[away_win, "winner"] = df.loc[away_win, "away_team_id"]
df.loc[away_win, "loser"]  = df.loc[away_win, "home_team_id"]

# ============================
# 3. チームごとの勝敗数を集計
# ============================

# 勝利数
wins = (
    df.dropna(subset=["winner"])
      .groupby(["game_year", "winner"])
      .size()
      .reset_index(name="wins")
      .rename(columns={"winner": "team_id"})
)

# 敗北数
losses = (
    df.dropna(subset=["loser"])
      .groupby(["game_year", "loser"])
      .size()
      .reset_index(name="losses")
      .rename(columns={"loser": "team_id"})
)

# 引き分け数（ホーム側）
draws_home = (
    df[df["is_draw"]]
      .groupby(["game_year", "home_team_id"])
      .size()
      .reset_index(name="draws")
      .rename(columns={"home_team_id": "team_id"})
)

# 引き分け数（アウェイ側）
draws_away = (
    df[df["is_draw"]]
      .groupby(["game_year", "away_team_id"])
      .size()
      .reset_index(name="draws")
      .rename(columns={"away_team_id": "team_id"})
)

# 引き分けを合算
draws = pd.concat([draws_home, draws_away], ignore_index=True)
draws = draws.groupby(["game_year", "team_id"])["draws"].sum().reset_index()

# ============================
# 4. 勝敗表を結合して勝率計算
# ============================

standings = (
    wins.merge(losses, on=["game_year", "team_id"], how="outer")
        .merge(draws, on=["game_year", "team_id"], how="outer")
)

# NaN → 0 に変換
standings = standings.fillna(0)

# 総試合数
standings["total_games"] = standings["wins"] + standings["losses"] + standings["draws"]

# 勝率（引き分けを勝ち0.5扱いにする場合は式を変更する）
standings["win_pct"] = standings["wins"] / (standings["wins"] + standings["losses"])

# ソート（勝率順）
standings = standings.sort_values(["game_year", "win_pct"], ascending=[True, False])

# ============================
# team_id → team_name の対応表を作る
# ============================

team_master_home = df[["home_team_id", "home_team_name"]] \
    .rename(columns={"home_team_id": "team_id", "home_team_name": "team_name"})

team_master_away = df[["away_team_id", "away_team_name"]] \
    .rename(columns={"away_team_id": "team_id", "away_team_name": "team_name"})

team_master = pd.concat([team_master_home, team_master_away]).drop_duplicates()

standings = standings.merge(team_master, on="team_id", how="left")

standings = standings[[
    "game_year", "team_id", "team_name",
    "wins", "losses", "draws", "total_games", "win_pct"
]]


# ============================
# 5. 結果表示
# ============================
print(standings)

# csvに保存
standings.to_csv("work_dataset/win_pct.csv", index=False, encoding="utf-8-sig")

    game_year  team_id team_name  wins  losses  draws  total_games   win_pct
0        2021        2      ヤクルト    79      54     19          152  0.593985
1        2021        5        阪神    77      58     10          145  0.570370
2        2021       11     オリックス    74      59     19          152  0.556391
3        2021        9       ロッテ    68      59     21          148  0.535433
4        2021      376        楽天    66      63     16          145  0.511628
5        2021        1        巨人    63      64     21          148  0.496063
6        2021       12    ソフトバンク    60      62     21          143  0.491803
7        2021        6        広島    63      68     12          143  0.480916
8        2021        8      日本ハム    55      68     20          143  0.447154
9        2021        7        西武    55      70     18          143  0.440000
10       2021        4        中日    55      71     17          143  0.436508
11       2021        3      DeNA    54      73     16          143  0.425197

In [16]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install matplotlib

  Using cached contourpy-1.3.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 1.5 MB/s  0:00:05 eta 0:00:01
Using cached contourpy-1.3.2-cp310-cp310-macosx_11_0_arm64.whl (253 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 2.4 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 745.9 kB/s  0:00:06 eta 0:00:01
Using cached pyparsing-3.2.5-py3-none-any.whl (113 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [matplotlib]7 [matplotlib]
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

# 1) データ読み込み
players = pd.read_csv("work_dataset/players.csv", encoding="cp932")
win = pd.read_csv("work_dataset/win_pct.csv")  # team_name, game_year, win_pct

# 2) チームごとの平均年俸
team_salary = (
    players
    .groupby("team_name", as_index=False)["salary"]
    .mean()
    .rename(columns={"salary": "avg_salary"})
)

# 3) 結合
data = win.merge(team_salary, on="team_name", how="inner")

# 4) チームごと × 年ごとに比較
for team in sorted(data["team_name"].unique()):
    print(f"\n######## {team} ########")
    d_team = data[data["team_name"] == team].copy()

    # 年ごとの一覧
    print(d_team[["game_year", "avg_salary", "win_pct"]].sort_values("game_year"))

    # 年ごとの相関・回帰（平均年俸はチーム内で一定なら、ここは年跨ぎで見る）
    # チーム内で年俸が年ごとに違うならこのまま効く
    if d_team["avg_salary"].nunique() > 1:
        # 相関
        corr = d_team[["avg_salary", "win_pct"]].corr().loc["avg_salary", "win_pct"]
        print(f"平均年俸と勝率の相関係数: {corr:.3f}")

        # 回帰
        X = add_constant(d_team["avg_salary"])
        y = d_team["win_pct"]
        model = OLS(y, X).fit()
        coef = model.params["avg_salary"]
        pval = model.pvalues["avg_salary"]
        r2 = model.rsquared
        print(f"回帰係数(平均年俸): {coef:.6f}")
        print(f"p値: {pval:.4f}")
        print(f"決定係数 R^2: {r2:.3f}")
    else:
        print("このチームは年ごとの平均年俸が1種類しかないため、回帰・相関は計算できません。")


######## DeNA ########
    game_year   avg_salary   win_pct
11       2021  5297.317784  0.425197
15       2022  5297.317784  0.513889
27       2023  5297.317784  0.521127
40       2024  5297.317784  0.525974
52       2025  5297.317784  0.514085
このチームは年ごとの平均年俸が1種類しかないため、回帰・相関は計算できません。

######## オリックス ########
    game_year   avg_salary   win_pct
2        2021  4437.520349  0.556391
13       2022  4437.520349  0.549669
25       2023  4437.520349  0.613333
44       2024  4437.520349  0.450000
51       2025  4437.520349  0.521127
このチームは年ごとの平均年俸が1種類しかないため、回帰・相関は計算できません。

######## ソフトバンク ########
    game_year  avg_salary   win_pct
6        2021      9220.0  0.491803
14       2022      9220.0  0.537415
29       2023      9220.0  0.503497
36       2024      9220.0  0.644295
48       2025      9220.0  0.626667
このチームは年ごとの平均年俸が1種類しかないため、回帰・相関は計算できません。

######## ヤクルト ########
    game_year   avg_salary   win_pct
0        2021  5494.364162  0.593985
12       2022  5494.364162  0.574324
34       2

In [13]:
import pandas as pd
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

# 1) データ読み込み
players = pd.read_csv("work_dataset/players.csv", encoding="cp932")   # team_name, salary
win = pd.read_csv("work_dataset/win_pct.csv")       # team_name, game_year, win_pct

# 2) チームごとの平均年俸を計算
team_salary = (
    players
    .groupby("team_name", as_index=False)["salary"]
    .mean()
    .rename(columns={"salary": "avg_salary"})
)

# 3) 勝率データと平均年俸を結合（年も残す）
data = win.merge(team_salary, on="team_name", how="inner")

# 4) 年ごとに「平均年俸と勝率」をざっくり見る
for year in sorted(data["game_year"].unique()):
    print(f"\n===== {year}年 =====")
    d = data[data["game_year"] == year]

    # チームごとの一覧
    print(d[["team_name", "avg_salary", "win_pct"]].sort_values("avg_salary"))

    # 相関係数
    corr = d[["avg_salary", "win_pct"]].corr().loc["avg_salary", "win_pct"]
    print(f"平均年俸と勝率の相関係数: {corr:.3f}")

    # 単回帰（平均年俸 → 勝率）
    X = add_constant(d["avg_salary"])
    y = d["win_pct"]
    model = OLS(y, X).fit()

    # 回帰の重要な結果だけ表示
    coef = model.params["avg_salary"]
    pval = model.pvalues["avg_salary"]
    r2 = model.rsquared
    print(f"回帰係数(平均年俸): {coef:.6f}")
    print(f"p値: {pval:.4f}  （0.05より小さいと“効いてそう”な目安）")
    print(f"決定係数 R^2: {r2:.3f}")


===== 2021年 =====
   team_name   avg_salary   win_pct
8       日本ハム  3970.494350  0.447154
10        中日  4038.664740  0.436508
7         広島  4130.695652  0.480916
9         西武  4326.962209  0.440000
2      オリックス  4437.520349  0.556391
3        ロッテ  4497.742857  0.535433
1         阪神  5027.385714  0.570370
11      DeNA  5297.317784  0.425197
4         楽天  5364.794118  0.511628
0       ヤクルト  5494.364162  0.593985
5         巨人  6972.779370  0.496063
6     ソフトバンク  9220.000000  0.491803
平均年俸と勝率の相関係数: 0.122
回帰係数(平均年俸): 0.000005
p値: 0.7060  （0.05より小さいと“効いてそう”な目安）
決定係数 R^2: 0.015

===== 2022年 =====
   team_name   avg_salary   win_pct
23      日本ハム  3970.494350  0.421429
22        中日  4038.664740  0.468085
21        広島  4130.695652  0.471429
16        西武  4326.962209  0.507042
13     オリックス  4437.520349  0.549669
18       ロッテ  4497.742857  0.485915
20        阪神  5027.385714  0.482759
15      DeNA  5297.317784  0.513889
17        楽天  5364.794118  0.492857
12      ヤクルト  5494.364162  0.574324
19    

In [14]:
import pandas as pd
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant
import matplotlib.pyplot as plt

# 1) データ読み込み
players = pd.read_csv("work_dataset/players.csv", encoding="cp932")
win = pd.read_csv("work_dataset/win_pct.csv")  # team_name, game_year, win_pct

# 2) チームごとの平均年俸
team_salary = (
    players
    .groupby("team_name", as_index=False)["salary"]
    .mean()
    .rename(columns={"salary": "avg_salary"})
)

# 3) 結合
data = win.merge(team_salary, on="team_name", how="inner")

# 4) 年ごとに分析（相関係数・単回帰・散布図）
for year in sorted(data["game_year"].unique()):
    print(f"\n===== {year}年 =====")
    d = data[data["game_year"] == year]

    # 一覧
    print(d[["team_name", "avg_salary", "win_pct"]].sort_values("avg_salary"))

    # 4-1 相関係数
    corr = d[["avg_salary", "win_pct"]].corr().loc["avg_salary", "win_pct"]
    print(f"平均年俸と勝率の相関係数: {corr:.3f}")

    # 4-2 単回帰
    X = add_constant(d["avg_salary"])
    y = d["win_pct"]
    model = OLS(y, X).fit()
    coef = model.params["avg_salary"]
    pval = model.pvalues["avg_salary"]
    r2 = model.rsquared
    print(f"回帰係数(平均年俸): {coef:.6f}")
    print(f"p値: {pval:.4f}  （0.05より小さいと“効いてそう”な目安）")
    print(f"決定係数 R^2: {r2:.3f}")

    # 4-3 散布図（回帰直線付き）
    plt.figure()
    plt.scatter(d["avg_salary"], d["win_pct"], label="team")
    # 回帰直線
    x_line = pd.Series(sorted(d["avg_salary"]))
    y_line = model.params["const"] + model.params["avg_salary"] * x_line
    plt.plot(x_line, y_line, color="red", label="regression line")

    plt.xlabel("平均年俸")
    plt.ylabel("勝率")
    plt.title(f"{year}年 平均年俸と勝率の関係")
    plt.legend()
    plt.tight_layout()
    plt.show()

ModuleNotFoundError: No module named 'matplotlib'